In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
import cupy as cp
from PIL import Image


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
def sigmoid(x):
    return 1 / (1 + cp.exp(-x))

def softmax(x):
    exps = cp.exp(x - cp.max(x))
    return exps / cp.sum(exps, axis=1, keepdims=True)

def sigmoid_derivative(x):
    return x * (1 - x)

In [3]:
def cross_entropy_loss(y_true, y_pred):
    n_samples = y_true.shape[0]
    logp = - cp.log(y_pred[range(n_samples), y_true.argmax(axis=1)])
    loss = cp.sum(logp) / n_samples
    return loss

In [4]:
def one_hot(y, num_classes):
    if y.ndim > 1:  # Flatten the array if necessary
        y = y.flatten()
        
    one_hot_labels = cp.zeros((y.shape[0], num_classes))
    one_hot_labels[cp.arange(y.shape[0]), y] = 1
    return one_hot_labels

In [5]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        self.input_size = input_size
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.output_size = output_size

        # Initialize weights and biases
        self.W1 = cp.random.randn(input_size, hidden_size1) * 0.01
        self.b1 = cp.zeros((1, hidden_size1))
        self.W2 = cp.random.randn(hidden_size1, hidden_size2) * 0.01
        self.b2 = cp.zeros((1, hidden_size2))
        self.W3 = cp.random.randn(hidden_size2, output_size) * 0.01
        self.b3 = cp.zeros((1, output_size))

    def forward(self, X):
        self.z1 = cp.dot(X, self.W1) + self.b1
        self.a1 = sigmoid(self.z1)
        self.z2 = cp.dot(self.a1, self.W2) + self.b2
        self.a2 = sigmoid(self.z2)
        self.z3 = cp.dot(self.a2, self.W3) + self.b3
        self.a3 = softmax(self.z3)
        return self.a3

    def backward(self, X, y_true, learning_rate):
        m = y_true.shape[0]
        y_pred = self.a3

        # Calculate gradients
        dz3 = y_pred - y_true
        dW3 = cp.dot(self.a2.T, dz3) / m
        db3 = cp.sum(dz3, axis=0, keepdims=True) / m

        dz2 = cp.dot(dz3, self.W3.T) * sigmoid_derivative(self.a2)
        dW2 = cp.dot(self.a1.T, dz2) / m
        db2 = cp.sum(dz2, axis=0, keepdims=True) / m

        dz1 = cp.dot(dz2, self.W2.T) * sigmoid_derivative(self.a1)
        dW1 = cp.dot(X.T, dz1) / m
        db1 = cp.sum(dz1, axis=0, keepdims=True) / m

        # Update weights and biases
        self.W3 -= learning_rate * dW3
        self.b3 -= learning_rate * db3
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1

    def train(self, X, y, epochs, learning_rate):
        y_one_hot = one_hot(y, self.output_size)

        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = cross_entropy_loss(y_one_hot, y_pred)
            self.backward(X, y_one_hot, learning_rate)

            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')

    def predict(self, X):
        y_pred = self.forward(X)
        return np.argmax(y_pred, axis=1)

In [6]:
train_data = pd.read_csv('train_data.csv')
train_data = train_data.to_numpy()
    
#Cortar en features y labales
train_samples = train_data.shape[0]
features = train_data[:train_samples, 1:-1]  # Features for training    
labels = train_data[:train_samples, -1]   # Labels for training

features = cp.array(features)
labels = cp.array(labels, ndmin=2)
labels = labels.reshape(-1, 1)  # Reshape to (299, 1)

indices = cp.arange(train_samples)

X_train = cp.array(features)
y_train = cp.array(labels).flatten()

red_pixels = []
green_pixels = []
blue_pixels = []
train_data = train_data[:, 1:-1]

# Iterate through each row in the original matrix
for row in train_data:
    pr = []
    pg = []
    pb = []
    for i in range(len(row)):
        if i % 3 == 0:
            pr.append(row[i])
        
        if i % 3 == 1:
            pg.append(row[i])

        if i % 3 == 2:
            pb.append(row[i])

    red_pixels.append(pr)
    green_pixels.append(pg)
    blue_pixels.append(pb)

#intensidad media por conal
mean_red = np.mean(red_pixels, axis=1)
mean_green = np.mean(green_pixels, axis=1)
mean_blue = np.mean(blue_pixels, axis=1)

#varianza
var_red = np.var(red_pixels, axis=1)
var_green = np.var(green_pixels, axis=1)
var_blue = np.var(blue_pixels, axis=1)

#desviación estándar
std_red = np.std(red_pixels, axis=1)
std_green = np.std(green_pixels, axis=1)
std_blue = np.std(blue_pixels, axis=1)

#contraste
contrast_red = np.ptp(red_pixels, axis=1) # ptp: peak to peak (max - min)
contrast_green = np.ptp(green_pixels, axis=1)
contrast_blue = np.ptp(blue_pixels, axis=1)

channel_data = np.column_stack((
                                mean_red, mean_green, mean_blue,
                                var_red, var_green, var_blue,
                                std_red, std_green, std_blue,
                                contrast_red, contrast_green, contrast_blue
                                ))

print(channel_data.shape)

#Cortar en features y labales
channel_data_train_samples = channel_data.shape[0]
features = channel_data  # Features for training

features = cp.array(features)

print(channel_data.shape)
print(features.shape)
print(labels.shape)

X_train = cp.array(features)

In [10]:
# Hyperparameters
input_size = 3072
hidden_size1 = 256
hidden_size2 = 256
output_size = 10
epochs = 1000
learning_rate = 0.01

# Initialize and train the neural network
nn = NeuralNetwork(input_size, hidden_size1, hidden_size2, output_size)
nn.train(X_train, y_train, epochs, learning_rate)

# Make predictions
predictions = nn.predict(X_train)
print(predictions)

accuracy = cp.mean(predictions == y_train)
print(f'Test Accuracy: {accuracy}')

Epoch 0, Loss: 2.3045199373070764
Epoch 100, Loss: 2.289785783213048
Epoch 200, Loss: 2.1926823025760247
[6 6 8 ... 6 6 0]
Test Accuracy: 0.19154901960784312
